## CDD 단어 추출
- CDD에서 정의된 용어, 단어는 검증이 완료되었다고 판단
- 각 데이터 에서 단어를 최대한 많이 뽑아내는 것이 목적

### CDD class data
- 정리된 CDD class excel 파일 3212 개 
- Class단위의 PreferredName에서 단어 추출

In [2]:
#라이브러리 import

import pandas as pd
from openpyxl import load_workbook
import os
import re
from collections import Counter

dir_path = os.getcwd()
dir_path = os.path.join(dir_path, "file")
print(dir_path)

c:\Users\PC\Desktop\git_repo\ML-DL-Training\Data_preprocess\workspace\file


### 1차 데이터 처리 작업

- CDD 클래스 정리 파일 로딩
- 토큰화를 통한 단어 분리
- 특수 문자 제거 및 숫자 제거
- 불용어 제거, 길이가 2이하인 단어 제거

In [3]:
# 파일 경로와 읽을 열 이름
file_name = "CDD_class.xlsx"
file_path = os.path.join(dir_path,file_name)  
column_name = 'PreferredName'  

# 엑셀 파일 읽기
df = pd.read_excel(file_path)

In [4]:
#nltk 라이브러리 import

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

#불용어
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


- 각 기능별 함수 정의

In [5]:

# 품사태그를 WordNet의 태그로 변환
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None

# Definition 문장 전처리 함수
def process_sentence(sentence):
    # 문장 토큰화
    tokens = word_tokenize(sentence)
    # 특수문자 제거
    tokens = [word for word in tokens if word.isalpha()]
    # 품사 태깅
    tagged_tokens = pos_tag(tokens)
    # 불용어 리스트 가져오기
    stop_words = set(stopwords.words('english'))
    # Lemmatizer 객체 생성
    lemmatizer = WordNetLemmatizer()
    # 불용어 제거 및 단어의 원형 추출
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag) or nltk.corpus.wordnet.NOUN)
        for word, tag in tagged_tokens if word.lower() not in stop_words and len(word) > 2
    ]
    return lemmatized_tokens

In [6]:
#각 column 데이터에 따른 데이터프레임 생성
def word_df_create(column_name):

    # 해당 열의 단어 추출 및 처리
    all_words = []
    for sentence in df[column_name].dropna():
        words = process_sentence(sentence)
        all_words.extend(words)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(set(all_words), columns=['Word'])

    return result_df    

In [7]:
#column_name: Preferred name -> pre_df 데이터 프레임 생성

pre_df = word_df_create(column_name)

In [8]:
pre_df

,Word
0,Proximity
1,Bus
2,Restriction
3,Circular
4,compensation
...,...
1202,cradle
1203,viscosity
1204,setpoint
1205,Installation


### 2차 데이터 처리 작업

- 대문자 중복 단어 제거
- 축약어, 단어 분리 처리
- 단어 표제어 변환
- 각 column 별 데이터 추출 후 병합, 중복 처리
- 최종 데이터 엑셀파일 저장   
 **-> sheet1: 단어 데이터, sheet2: 대문자 축약어 데이터**

In [9]:
#대문자 중복 단어 제거 및 축약어와 단어의 분리
def Cap_Dup_process(result_df):

    column_data = result_df['Word'].dropna()

    upper_data = pd.DataFrame()
    lower_data = pd.DataFrame()

    # pattern: 대문자 2개 이상 or 첫 문자가 소문자이면서 이외의 글자가 대문자인 경우
    pattern = r'^(?=.*[A-Z].*[A-Z])|^(?:[a-z].*[A-Z])'

    # 패턴의 포함 여부에 따라 분류
    upper_data['Word'] = column_data[column_data.str.contains(pattern, regex=True)]             
    lower_data['Word'] = column_data[~column_data.str.contains(pattern, regex=True, na=False)]  

    # 소문자 통일 처리
    upper_data['UP_Lower'] = upper_data['Word'].str.lower().to_frame()
    lower_data['LO_Lower'] = lower_data['Word'].str.lower().to_frame()

    # lower_data를 기준으로 inner merge
    merged_df = pd.merge(upper_data, lower_data, left_on='UP_Lower', right_on='LO_Lower', how='inner')

    # upper_data 중 중복이 아닌 데이터 분리
    upper_only_data = upper_data.loc[~upper_data['UP_Lower'].isin(merged_df['UP_Lower']), ['Word']] 

    # lower_data 중 중복이 아닌 데이터 분리
    lower_only_data = lower_data.loc[~lower_data['LO_Lower'].isin(merged_df['LO_Lower']), ['Word']] 

    # overlap data 정리
    overlap_data = merged_df[['Word_y']].rename(columns={'Word_y': 'Word'})                        

    # word data 정리
    word_data = pd.concat([lower_only_data, overlap_data], ignore_index=True)                       
    word_data['Word'] = word_data['Word'].str.lower().drop_duplicates()

    # 결측치 처리 및 index 재정의
    return word_data.dropna().reset_index(drop=True), upper_only_data.reset_index(drop=True)

In [10]:
#각 column데이터 별로 단어 df, 대문자 df로 분리

pre_word_df, pre_upper_df = Cap_Dup_process(pre_df)

- 표제어 변환

In [11]:
from textblob import TextBlob, Word
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet, words

nltk.download('omw-1.4')
nltk.download('words')

EN_words = set(words.words())

import inflect

p = inflect.engine()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [21]:
# Steemmer를 사용하여 단어의 원형 추출
def stemmer(word):
    ps = PorterStemmer()
    return ps.stem(word)

# 단수형 변환 함수 정의
def to_singular(word):
    singular_word = p.singular_noun(word)
    return singular_word if singular_word else word

# 영어 단어 인지 확인
def is_EN_word(word):
    if word in EN_words:
        return word
    
lemmatizer = WordNetLemmatizer()

# 원형을 추출
def get_lemma(word):
    pos = pos_tag([word])[0][1]  # 단어의 품사 태깅
    wordnet_pos = get_wordnet_pos(pos)  # WordNet의 품사 태그로 변환
    lemma = lemmatizer.lemmatize(word, wordnet_pos)  # 원형 추출
    return lemma


# 2이하인 값 제거
def remove_short_words(cell):
    return cell if len(cell) > 2 else None

In [22]:
# 표제어 추출
def Lemmatization_process(word_data):
    # 데이터프레임에 단어의 원형 추출
    word_result = pd.DataFrame()

    # 추출 단어의 단수형 추출, nltk 내의 word에서 존재여부 파악, 중복 처리
    word_result['Word'] = word_data['Word'].apply(lambda x: is_EN_word(to_singular(x))).dropna().drop_duplicates()

    # 추출 단어의 어간 추출(Stemming)
    word_result['Stemm_Word'] = word_result['Word'].apply(stemmer)

    # 어간 추출 단어와 기존 단어의 비교
    word_result['Data_Equal'] = word_result.eval('Word == Stemm_Word')

    # 어간 추출 단어와 다른 단어중 nltk 내의 word에서 존재여부 파악 
    Stemm_Word = word_result[word_result['Data_Equal'] == False][['Stemm_Word']]
    word_result['Stem_con'] = Stemm_Word['Stemm_Word'].apply(is_EN_word)

    # 최종 결과 col생성
    word_result['Result_word'] = word_result.apply(
        lambda row: row['Word'] if row['Data_Equal'] else (row['Stemm_Word'] if row['Stem_con'] else row['Word']),axis=1
    )

    # 2이하인 값 제거
    word_result['Result_word'] = word_result['Result_word'].apply(remove_short_words).dropna()

    return word_result['Result_word'].drop_duplicates().to_frame(name='Word').reset_index(drop=True)

In [27]:
# 최종 단어 추출
word_result = Lemmatization_process(pre_word_df).dropna()

upper_result = pre_upper_df

In [28]:
word_result

,Word
0,proximity
2,restrict
3,circular
4,compensation
5,attach
...,...
762,cradle
763,viscosity
764,input
765,output


In [29]:
upper_result

,Word
0,SPD
1,READBACK
2,DLOP
3,PTC
4,LPG
5,HART
6,SIMULATE
7,LED
8,RTD
9,DIP


In [30]:
# 결과 엑셀 시트에 분할 저장

result_file_path = os.path.join(dir_path,'CDD_Word_Data.xlsx')
with pd.ExcelWriter(result_file_path) as writer:
    word_result.to_excel(writer, sheet_name='Sheet1', index=False)
    upper_result.to_excel(writer, sheet_name='Sheet2', index=False)